In [1]:
import pymongo
import pandas as pd
import numpy as np
import re
import time
import datetime
import nltk
import praw
from datetime import datetime, timedelta
from pymongo import MongoClient
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bson.objectid import ObjectId

In [2]:
def getContinuousChunks(text):
    chunked = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))
    continuousChunkdf = pd.DataFrame(columns = ['id', 'Named Entity', 'Label'])
    currentChunk = []
    currentLabel = []
    for i in chunked:
        if type(i) == nltk.tree.Tree:
            currentChunk.append(" ".join([token for token, pos in i.leaves()]))
            currentLabel.append(i.label())
        if currentChunk:
            namedEntity = " ".join(currentChunk)
            label = " ".join(currentLabel)
            if namedEntity not in continuousChunkdf['Named Entity']:
                d = {'id': 0, 'Named Entity': namedEntity, 'Label': label}
                continuousChunkdf = continuousChunkdf.append(d, ignore_index = True)
                currentChunk = []
                currentLabel = []
        else:
            continue
    return continuousChunkdf

def mapResults(result, leftNames, rightNames, threadID, threshold):
    result[result < threshold] = 0
    matchdf = pd.DataFrame(0, index = np.arange(len(result.nonzero()[0])), columns = ['id','Left', 'Right', 'Similarity'])
    for i in range(len(result.nonzero()[0])):
        matchdf.loc[i, 'Left'] = leftNames[result.nonzero()[0][i]]
        matchdf.loc[i, 'Right'] = rightNames[result.nonzero()[1][i]]
        matchdf.loc[i, 'Similarity'] = result[result.nonzero()[0][i]][result.nonzero()[1][i]]
    matchdf['id'] = threadID
    return matchdf.drop_duplicates(subset = 'Right')

def pullTickers(string):
    dollarTicker = set(re.findall(r"\$\b[A-Z]{1,4}\b",string))
    manualTicker = re.findall(r"\b[A-Z]{2,4}\b",string)
    manualTicker = list(set(manualTicker).difference(notTickers))
    manualTicker = set(['$' + manualTicker for manualTicker in manualTicker])
    combined = manualTicker.union(dollarTicker)
    return combined.intersection(companydf['Manual'])

notTickers = {'DCF', 'IMO', 'CAN', 'MMS', 'ARE', 'CDC', 'NEW', 'LOVE', 'NYC', 'CASH', 'AI', 
'NAV', 'GOOD', 'DD', 'ATH', 'APPS', 'EDIT', 'WOW', 'PCB', 'UNIT', 'TA', 'VG', 'SELF', 'MR',
'RARE', 'ALEX', 'KEY', 'STIM', 'GO', 'SEE', 'CFO', 'CAL', 'REV', 'PE', 'CHI', 'EVE', 'PDT',
'CO', 'EV', 'TTM', 'EOD', 'AT', 'HUGE', 'ES', 'ONE', 'PT', 'CEO', 'ZEN', 'NOW', 'JAN', 'O',
'OR', 'PG', 'ROCK', 'FOUR', 'ONE', 'TWO', 'FIVE', 'SIX', 'NINE', 'TEN', 'ON', 'SU', 'XT',
'WELL', 'NOV', 'MAR', 'JAN', 'FUN', 'NOW', 'VERY', 'USA', 'POST'}

In [2]:
client = MongoClient('localhost', 27017)
db = client.db
threads = db.threads
companies = db.companylist
secret = db.secret.find_one()['Secret']

reddit = praw.Reddit(client_id='-wpcPIbA7bhlpw', client_secret=secret, user_agent='sentiment')
subreddit = reddit.subreddit('wallstreetbets')
postlist = []
commentlist = []

In [10]:
for post in subreddit.hot(limit = 100):
    print(post.title)
    postlist.append(post.comments)

Weekend Discussion Thread for the Weekend of October 30, 2020
Welcome to the Fund
This is the way.
Happy Halloween you bozos
We may hate each other, but together, we make the options market liquid 🇺🇸
how fucked are we boys
60000 SPY $290 Puts 11/4
Potential new collab?
Monday crash confirmed sell everything
RIP Monday
Which one of you retards was at my local brewery today?
Got an interesting DM today, Had to show her the way to tendies.
20 years old, working to pay for sister's 20k chemo bill. Said fuck it, the first one's free. But I'll bet that I'll be up next week too. Comment your wager. No pussies.
2020 stock market crash Wikipedia
Yall rn
Tried to conduct market research but I got made
Mr. Bezos, I don't feel so good
GME in defensive mode.
Get a load of these fa-
Am I doing this right?
!!!
Every time I post in r/Politics I'm called out for being a Wallstreetbets user.
Calls on STRB Monday, you retards
The OG
See you guys next year! Or maybe ill try Webull...
🌈🐻🌈🐻🌈🐻 LETS THE BEARV

In [11]:
from praw.models import MoreComments
for post in postlist:    
    for comment in post.list():
        commentlist.append(comment)

In [14]:
commentlistlen = 0
currentlen = len(commentlist)
while(commentlistlen != currentlen):
    commentlistlen = len(commentlist)
    for comment in commentlist:
        if isinstance(comment, MoreComments):
            commentlist.remove(comment)
    currentlen = len(commentlist)

In [15]:
len(commentlist)

6789

In [16]:
commentDf = pd.DataFrame(columns = ['Inserted Date','Body', 'Score', 'Stocks'])

In [17]:
created = [comment.created_utc for comment in commentlist]
bodies = [comment.body for comment in commentlist]
scores = [comment.score for comment in commentlist]

In [18]:
commentDf['Inserted Date'] = created
commentDf['Body'] = bodies
commentDf['Score'] = scores

In [19]:
commentDf

,Inserted Date,Body,Score,Stocks
0,1.604089e+09,"Next week: ""Stocks rally as civil war misses e...",258,NaN
1,1.604104e+09,"Guys I’m just so happy, I made 14K today, I ca...",121,NaN
2,1.604089e+09,Everything was fine up until Tesla battery day,158,NaN
3,1.604098e+09,I’m posting “ber r fuk” on Monday regardless o...,155,NaN
4,1.604097e+09,"Lost 45k today, now I’m gonna go beat my meat.",146,NaN
...,...,...,...,...
6784,1.604105e+09,"That's because there was a merger, I don't thi...",1,NaN
6785,1.604086e+09,Yeah I had no choice it expires today. I actua...,11,NaN
6786,1.604088e+09,MM wants to keep you playing so they can squee...,3,NaN
6787,1.604087e+09,11/6 puts at what strike?,2,NaN


In [20]:
def cleanStrings(string):
    return re.sub("[^a-zA-Z0-9./$:,'&]+", ' ',string) #only include normal string characters
def cleanText(text):
    return re.sub("http[s]?://\S+", ' ', text) #Remove links

In [21]:
commentDf['Body'] = commentDf['Body'].apply(cleanStrings)
commentDf['Body'] = commentDf['Body'].apply(cleanText)

In [22]:
companydf = pd.DataFrame.from_records(companies.find())

In [23]:
companyNames = companydf['Name'].unique()
companySymbols = companydf['Symbol'].unique()
nameVectorizer = TfidfVectorizer(min_df = 1)
symbolVectorizer = TfidfVectorizer(min_df = 1)
companyMatrix = nameVectorizer.fit_transform(companyNames)
symbolMatrix = symbolVectorizer.fit_transform(companySymbols)

In [24]:
commentDf['Stocks'] = commentDf['Stocks'].astype(object)

In [25]:
for i in range(len(commentDf['Body'])):
    sent = commentDf['Body'][i]
    threadID = i
    chunks = getContinuousChunks(sent)
    chunkdf = chunks.loc[(chunks['Label'] == 'ORGANIZATION') | (chunks['Label'] == 'PERSON')].reset_index()
    stocklist = []

    if len(chunkdf['Label']) > 0:
        nerNameMatrix = nameVectorizer.transform(chunkdf['Named Entity'])
        nerSymbolMatrix = symbolVectorizer.transform(chunkdf['Named Entity'])
        nameResult = cosine_similarity(nerNameMatrix, companyMatrix)
        symbolResult = cosine_similarity(nerSymbolMatrix, symbolMatrix)
        namedf = mapResults(nameResult, chunkdf['Named Entity'], companyNames, threadID, 0.85)
        symboldf = mapResults(symbolResult, chunkdf['Named Entity'], companySymbols, threadID, 1)
        stocklist = list(set(namedf.loc[namedf['Similarity'] > 0.8, 'Right'].append(symboldf.loc[symboldf['Similarity'] > 0.999, 'Right'])))
    commentDf.at[threadID, 'Stocks'] = stocklist

In [26]:
#commentDf = commentDf.drop(commentDf.loc[commentDf['Stocks'].str.len() == 0].index).reset_index()

In [27]:
#commentDf = commentDf.drop(columns = ['index'])

In [28]:
commentDf

,Inserted Date,Body,Score,Stocks
0,1.604089e+09,Next week: Stocks rally as civil war misses ex...,258,[]
1,1.604104e+09,"Guys I m just so happy, I made 14K today, I ca...",121,[]
2,1.604089e+09,Everything was fine up until Tesla battery day,158,[]
3,1.604098e+09,I m posting ber r fuk on Monday regardless of ...,155,[]
4,1.604097e+09,"Lost 45k today, now I m gonna go beat my meat.",146,[]
...,...,...,...,...
6784,1.604105e+09,"That's because there was a merger, I don't thi...",1,[]
6785,1.604086e+09,Yeah I had no choice it expires today. I actua...,11,[]
6786,1.604088e+09,MM wants to keep you playing so they can squee...,3,[]
6787,1.604087e+09,11/6 puts at what strike,2,[]
